In [ ]:
from pygame import *
from random import randint


scr = display.set_mode((280,550))
display.set_caption('MasterMind')

mmbg    = image.load('MMbg2.jpg')
palette = image.load('palette2.png')
myst    = image.load('myst.png')

palet_rect  = Rect(20,440,240,30)
go_rect = Rect(20,475,240,30)
reset_rect = Rect(20,510,240,30)

palet_mask = mask.from_surface(palette)

def play():
    
    combi = []
    ready = False
    
    def submit(value):
        try:
            a,b = zip(*[(a,b) for a,b in zip(combi,value) if a!=b])
            a   = list(a)
        except: return 4,0
        for i in b:
            try: a.remove(i)
            except: continue
        return 4-len(b),len(b)-len(a)
        
    scr.blit(mmbg,(0,0))
    scr.blit(palette,palet_rect)
    display.flip()
    while(len(combi) < 4 or ready != True):
        ev = event.wait()
        if ev.type == MOUSEBUTTONUP:
            if palet_rect.collidepoint(ev.pos):
                x,y = ev.pos
                x -= palet_rect.x
                y -= palet_rect.y
                if palet_mask.get_at((x,y)):
                    combi.append(x//30)
                    scr.blit(palette,(80+(len(combi)-1)*30,20),(x//30*30,0,30,30))
                    display.flip()
            if reset_rect.collidepoint(ev.pos):
                combi = []
                scr.blit(mmbg,(0,0))
                scr.blit(palette,palet_rect)
                display.flip()
            if go_rect.collidepoint(ev.pos):
                ready=True
                #deploy AI agent function here
                for ligne in range(10):
                    r=scr.blit(myst,(50,ligne*35+70))
                    display.update(r)
                    prediction = [randint(0,5),randint(0,5),randint(0,5),randint(0,5)]
                    for i in range(4):
                        r = scr.blit(palette, ((i+1)*30+20, ligne*35+70),(prediction[i]*30,0,30,30))
                        display.update(r)
                    placed,misplaced = submit(prediction)
                    for e,c in enumerate([(255,0,0)]*placed+[(255,255,255)]*misplaced):
                        r = draw.circle(scr,c,(190+e*10,ligne*35+85),2,0)
                        display.update(r)
                if placed == 4:
                    display.set_caption('GAGNE !')
                    break
                else:
                    display.set_caption('PERDU !')

play()
while 1:
    ev = event.wait()
    if ev.type == QUIT: break
    elif ev.type == MOUSEBUTTONUP: play()

    



    


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
